## 🧪 E3: Loss 가중치 조정 실험

> **목표**: class weight 조정으로 **실제 데이터 비율 반영** 및 **FP/FN 균형 최적화**

### 배경
- 실제 운영 환경에서 **욕설:정상 ≈ 1:4** 비율
- 악성(희소 클래스)에 더 높은 가중치를 줘야 균형 잡힌 학습 가능
- `metric_for_best_model = f1` → Precision/Recall 균형 최적화 (FP/FN 동시 감소)

### 실험 내용
- `CrossEntropyLoss`의 class weight 조정
- **w0(정상) ↑**: 정상→악성 오분류 벌점 증가 → FP 감소
- **w1(악성) ↑**: 악성→정상 오분류 벌점 증가 → FN 감소

### 변경 사항
- `metric_for_best_model`: f1 (FP/FN 균형)
- `class_weights`: **[w0, w1] 조정** ⭐
- `threshold`: 0.5 (기존 유지)

### 실험 버전
| 버전 | w0 | w1 | 설명 |
|------|-----|-----|------|
| **E3-R** (권장) | 1.0 | 4.0 | 실제 비율 반영 (악성 4배) |
| E3-R2 | 1.0 | 2.0 | 실제 비율 약하게 (악성 2배) |
| E3-1 | 1.5 | 0.5 | FP 감소 중심 |
| E3-2 | 2.0 | 0.5 | FP 강하게 감소 |
| E3-3 | 0.5 | 1.5 | FN 감소 중심 |
| E3-4 | 0.25 | 1.0 | FN 강하게 감소 |

In [ ]:
# ========================================
# 📦 패키지 설치 (필요시)
# ========================================
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
%pip install transformers==4.42.0 datasets accelerate scikit-learn pandas matplotlib seaborn -q

In [ ]:
# ========================================
# 🔧 경로 설정 (본인 환경에 맞게 수정)
# ========================================
import os

# 절대 경로 사용 (본인 환경에 맞게 수정하세요)
DATA_DIR = r"c:\00_Sookmyunguniv\DACOS(2025)\2학기\프로젝트\github\25-2-team3\data"
OUTPUT_DIR = r"./E3_output"

# 경로 확인
if not os.path.exists(DATA_DIR):
    print(f"⚠️ 데이터 폴더를 찾을 수 없습니다: {DATA_DIR}")
    print(f"   현재 작업 디렉토리: {os.getcwd()}")
else:
    print(f"📂 데이터 폴더: {DATA_DIR}")
print(f"📁 출력 폴더: {OUTPUT_DIR}")

In [ ]:
# ========================================
# ⭐ 실험 파라미터 설정 (여기서 조정!)
# ========================================

# 클래스 가중치 설정 (w0=정상, w1=악성)
# - w0 높이면 → FP(정상→악성 오탐) 벌점 증가
# - w1 높이면 → FN(악성→정상 미탐) 벌점 증가

# ===== 실험 버전 선택 (하나만 주석 해제) =====

# --- 🌟 실제 데이터 비율 반영 (권장) ---
# 실제 운영 환경: 욕설:정상 ≈ 1:4
# → 악성(희소)에 4배 가중치 = FP/FN 균형 최적화

# E3-R: 실제 비율 반영 (악성 4배)
W0, W1 = 1.0, 4.0
EXPERIMENT_ID = "E3-R"

# E3-R2: 실제 비율 약하게 (악성 2배)
# W0, W1 = 1.0, 2.0
# EXPERIMENT_ID = "E3-R2"

# --- FP(오탐) 감소 중심 ---
# E3-1: FP 감소 (정상 3배 중시)
# W0, W1 = 1.5, 0.5
# EXPERIMENT_ID = "E3-1"

# E3-2: FP 강하게 감소 (정상 4배 중시)
# W0, W1 = 2.0, 0.5
# EXPERIMENT_ID = "E3-2"

# --- FN(미탐) 감소 중심 ---
# E3-3: FN 감소 (악성 3배 중시)
# W0, W1 = 0.5, 1.5
# EXPERIMENT_ID = "E3-3"

# E3-4: FN 강하게 감소 (악성 4배 중시) 
# W0, W1 = 0.25, 1.0
# EXPERIMENT_ID = "E3-4"

# =============================================

print(f"🔬 실험 ID: {EXPERIMENT_ID}")
print(f"⚖️ Class weights: [w0={W0}, w1={W1}]")
ratio = W1 / W0 if W0 > 0 else float('inf')
if ratio > 1:
    print(f"   → 악성 {ratio:.1f}배 중시 (FN 감소, 실제 비율 반영)")
elif ratio < 1:
    print(f"   → 정상 {1/ratio:.1f}배 중시 (FP 감소)")
else:
    print(f"   → 균형 (1:1)")

In [ ]:
# ========================================
# 1) 환경 준비
# ========================================
import os
import random
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (precision_recall_fscore_support, accuracy_score, 
                             roc_auc_score, confusion_matrix)
from datasets import Dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          Trainer, TrainingArguments)

# 시드 고정
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    print(f"✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU가 감지되지 않았습니다. CPU로 실행됩니다.")

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# ========================================
# 2) 데이터 로드
# ========================================
abusive_path = os.path.join(DATA_DIR, "combined_abusive_shuffled_20k.csv")
nonabusive_path = os.path.join(DATA_DIR, "nonabusive_merged_shuffled_sample20000.csv")

df_ab = pd.read_csv(abusive_path)[["text", "label"]]
df_non = pd.read_csv(nonabusive_path)[["text", "label"]]

df = pd.concat([df_ab, df_non], ignore_index=True).dropna().reset_index(drop=True)

print(f"📊 라벨 분포:\n{df['label'].value_counts()}")
print(f"\n총 데이터 수: {len(df)}개")

In [ ]:
# ========================================
# 3) Train/Test 분리
# ========================================
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["label"]
)
print(f"Train: {len(train_df)}, Test: {len(test_df)}")

In [ ]:
# ========================================
# 4) 토큰화
# ========================================
MODEL_ID = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

raw_ds = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(test_df),
})

tokenized = raw_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format("torch")

print("✅ 토큰화 완료!")

In [ ]:
# ========================================
# 5) ⭐ 커스텀 클래스 가중치 설정
# ========================================
# 위에서 설정한 W0, W1 사용
class_weights = torch.tensor([W0, W1], dtype=torch.float)

print(f"⭐ 커스텀 Class weights: {class_weights}")
print(f"   정상(0) 가중치: {W0} → FP 벌점 강화")
print(f"   악성(1) 가중치: {W1}")

In [ ]:
# ========================================
# 6) 모델 + Trainer 설정
# ========================================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=0)
    
    # Confusion matrix for FP
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    try:
        probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
        auc = roc_auc_score(labels, probs)
    except:
        auc = float("nan")
    
    return {
        "accuracy": acc, 
        "precision": prec, 
        "recall": rec, 
        "f1": f1, 
        "roc_auc": auc,
        "FP": fp,
        "FN": fn,
        "FPR": fpr,
    }

base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=2)

# ⭐ 핵심: 커스텀 가중치 적용
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # ⭐ 커스텀 가중치 사용
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

os.makedirs(OUTPUT_DIR, exist_ok=True)

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 기존 유지
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=base_model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("✅ Trainer 설정 완료!")
print(f"⭐ class_weights = [{W0}, {W1}]")

In [ ]:
# ========================================
# 7) 학습 실행
# ========================================
trainer.train()

In [ ]:
# ========================================
# 8) 최종 평가
# ========================================
final_metrics = trainer.evaluate()

print("\n" + "="*60)
print(f"📊 {EXPERIMENT_ID} 최종 평가 결과 (weight=[{W0},{W1}])")
print("="*60)
for key, value in final_metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
# ========================================
# 📝 9) 실험 결과 기록
# ========================================
print("\n" + "="*60)
print("📝 실험 결과 기록 (복사용)")
print("="*60)

# 변수 미리 포맷팅
fpr_str = f"{final_metrics['eval_FPR']:.4f}" if final_metrics.get('eval_FPR') is not None else 'N/A'
prec_str = f"{final_metrics['eval_precision']:.4f}" if final_metrics.get('eval_precision') is not None else 'N/A'
rec_str = f"{final_metrics['eval_recall']:.4f}" if final_metrics.get('eval_recall') is not None else 'N/A'
f1_str = f"{final_metrics['eval_f1']:.4f}" if final_metrics.get('eval_f1') is not None else 'N/A'
auc_str = f"{final_metrics['eval_roc_auc']:.4f}" if final_metrics.get('eval_roc_auc') is not None else 'N/A'

# 코멘트 동적 생성
if W0 > W1:
    comment = f"정상 가중치 {W0}로 FP(오탐) 벌점 강화\n  - 기존 대비 FP ↓, FN ↑ 예상"
elif W1 > W0:
    comment = f"악성 가중치 {W1}로 FN(미탐) 벌점 강화\n  - 기존 대비 FN ↓, FP ↑ 예상"
else:
    comment = "균형 가중치"

print(f"""
#### 실험 ID: {EXPERIMENT_ID}
- 변경 사항:
  - class weight: [{W0}, {W1}]
  - metric_for_best_model: f1 (기존 유지)
  - threshold: 0.5 (기존 유지)
- 데이터:
  - train/validation split: 0.8/0.2 (random_state=42, stratify=label)
  - max_length=128, batch_size=32, epochs=3
- Validation 성능:
  - FP(정상→악성): {final_metrics.get('eval_FP', 'N/A')}
  - FN(악성→정상): {final_metrics.get('eval_FN', 'N/A')}
  - FPR: {fpr_str}
  - Precision: {prec_str}
  - Recall: {rec_str}
  - F1: {f1_str}
  - AUC: {auc_str}
- 코멘트:
  - {comment}
""")

In [ ]:
# ========================================
# 💾 10) 모델 저장
# ========================================
SAVE_DIR = os.path.join(OUTPUT_DIR, f"best_model_{EXPERIMENT_ID}")
os.makedirs(SAVE_DIR, exist_ok=True)
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print(f"✅ 모델 저장 완료: {SAVE_DIR}")

---
## 📋 실험 버전 비교 가이드

| 버전 | w0 (정상) | w1 (악성) | 정상:악성 비율 | 예상 효과 |
|------|----------|----------|--------------|----------|
| 기존 | ~0.86 | ~1.14 | 0.75:1 | 악성 중시 |
| E3-1 | 1.5 | 0.5 | 3:1 | FP ↓↓, FN ↑ |
| E3-2 | 2.0 | 0.5 | 4:1 | FP ↓↓↓, FN ↑↑ |
| E3-3 | 1.2 | 0.8 | 1.5:1 | 약한 조정 |

### 권장 순서
1. E3-3 먼저 실행 (약한 조정)
2. FP 감소 효과 확인 후 E3-1 시도
3. 필요시 E3-2 (강한 조정)

### 주의사항
- w0를 너무 높이면 악성 댓글을 놓칠 수 있음 (FN 증가)
- 운영 목적에 맞게 FP vs FN 트레이드오프 고려